In [21]:
import pandas as pd
import numpy as np
import dash
from dash import html, dcc, dash_table, Input, Output
import plotly.express as px

# Generate sample dataset
np.random.seed(0)
n_students = 120
names = [f'Student_{i}' for i in range(1, n_students + 1)]
roll_numbers = np.arange(1, n_students + 1)
classes = np.random.choice(['9A', '9B', '9C', '9D'], size=n_students)
attendance = np.random.randint(70, 100, size=n_students)
maths_marks = np.random.randint(20, 100, size=n_students)
science_marks = np.random.randint(20, 100, size=n_students)
english_marks = np.random.randint(20, 100, size=n_students)

df = pd.DataFrame({
    'Student Name': names,
    'Roll Number': roll_numbers,
    'Class': classes,
    'Attendance': attendance,
    'Marks in Maths': maths_marks,
    'Marks in Science': science_marks,
    'Marks in English': english_marks
})

df['Percentage'] = df[['Marks in Maths', 'Marks in Science', 'Marks in English']].mean(axis=1)

def assign_grade(percentage):
    if percentage >= 90:
        return 'A+'
    elif percentage >= 80:
        return 'A'
    elif percentage >= 70:
        return 'B'
    elif percentage >= 50:
        return 'C'
    else:
        return 'D'

def assign_status(percentage):
    return 'Pass' if percentage >= 50 else 'Fail'

df['Grade'] = df['Percentage'].apply(assign_grade)
df['Status'] = df['Percentage'].apply(assign_status)

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(" Student Dashboard", style={'textAlign': 'center', 'color':'green'}),
    
    # Filters
    html.Div([
        dcc.Dropdown(
            id='class-filter',
            options=[{'label': cls, 'value': cls} for cls in sorted(df['Class'].unique())],
            placeholder="Select Class"
        ),
        dcc.Dropdown(
            id='grade-filter',
            options=[{'label': grade, 'value': grade} for grade in sorted(df['Grade'].unique())],
            placeholder="Select Grade"
        ),
        dcc.Dropdown(
            id='status-filter',
            options=[{'label': status, 'value': status} for status in sorted(df['Status'].unique())],
            placeholder="Select Status"
        )
    ], style={'display': 'flex', 'gap': '10px', 'margin-bottom': '20px'}),

    # Summary Cards
    html.Div([
        html.Div(id='total-students', style={'padding': '10px', 'backgroundColor': '#e0f7fa', 'borderRadius': '10px'}),
        html.Div(id='pass-rate', style={'padding': '10px', 'backgroundColor': '#e8f5e9', 'borderRadius': '10px'}),
        html.Div(id='average-percentage', style={'padding': '10px', 'backgroundColor': '#fff3e0', 'borderRadius': '10px'})
    ], style={'display': 'flex', 'gap': '20px'}),

    # Leaderboard Table
    dash_table.DataTable(
        id='leaderboard-table',
        columns=[{'name': col, 'id': col} for col in ['Student Name', 'Roll Number', 'Class', 'Percentage', 'Grade', 'Status']],
        page_size=10,
        style_data_conditional=[
            {'if': {'filter_query': '{Percentage} >= 90'}, 'backgroundColor': '#d0f0c0'},
            {'if': {'filter_query': '{Percentage} < 50'}, 'backgroundColor': '#ffcdd2'}
        ],
        style_table={'overflowX': 'auto'}
    ),

    # Charts Section
    dcc.Graph(id='grade-distribution'),
    dcc.Graph(id='class-performance'),
    dcc.Graph(id='attendance-vs-percentage'),
    dcc.Graph(id='subject-distribution'),
    dcc.Graph(id='failures-by-class'),
    dcc.Graph(id='status-distribution'),
    dcc.Graph(id='percentage-trend'),
    dcc.Graph(id='subject-performance-box')
])

@app.callback(
    [Output('leaderboard-table', 'data'),
     Output('total-students', 'children'),
     Output('pass-rate', 'children'),
     Output('average-percentage', 'children'),
     Output('grade-distribution', 'figure'),
     Output('class-performance', 'figure'),
     Output('attendance-vs-percentage', 'figure'),
     Output('subject-distribution', 'figure'),
     Output('failures-by-class', 'figure'),
     Output('status-distribution', 'figure'),
     Output('percentage-trend', 'figure'),
     Output('subject-performance-box', 'figure')],
    [Input('class-filter', 'value'),
     Input('grade-filter', 'value'),
     Input('status-filter', 'value')]
)
def update_dashboard(selected_class, selected_grade, selected_status):
    filtered_df = df.copy()

    if selected_class:
        filtered_df = filtered_df[filtered_df['Class'] == selected_class]
    if selected_grade:
        filtered_df = filtered_df[filtered_df['Grade'] == selected_grade]
    if selected_status:
        filtered_df = filtered_df[filtered_df['Status'] == selected_status]

    total_students = f"Total Students: {len(filtered_df)}"
    pass_rate = f"Pass Rate: {filtered_df[filtered_df['Status'] == 'Pass'].shape[0] / len(filtered_df) * 100:.2f}%"
    average_percentage = f"Average Percentage: {filtered_df['Percentage'].mean():.2f}%"

    grade_fig = px.pie(filtered_df, names='Grade', title='Grade Distribution')
    class_fig = px.bar(filtered_df, x='Class', y='Percentage', color='Grade', title='Class-wise Performance')
    attendance_fig = px.scatter(filtered_df, x='Attendance', y='Percentage', color='Grade', title='Attendance vs Percentage')
    subject_fig = px.histogram(filtered_df, x=['Marks in Maths', 'Marks in Science', 'Marks in English'], barmode='overlay', title='Subject Marks Distribution')
    failures_fig = px.bar(filtered_df[filtered_df['Status'] == 'Fail'], x='Class', title='Failures by Class')
    status_fig = px.pie(filtered_df, names='Status', title='Status Distribution')
    trend_fig = px.line(filtered_df, x='Roll Number', y='Percentage', title='Percentage Trend over Roll Numbers')
    box_fig = px.box(filtered_df.melt(id_vars=['Student Name'], value_vars=['Marks in Maths', 'Marks in Science', 'Marks in English']), x='variable', y='value', title='Subject Performance Spread')

    return filtered_df.to_dict('records'), total_students, pass_rate, average_percentage, grade_fig, class_fig, attendance_fig, subject_fig, failures_fig, status_fig, trend_fig, box_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import pandas as pd
import numpy as np
import dash
from dash import html, dcc, dash_table, Input, Output
import plotly.express as px

# Load dataset from CSV file
df = pd.read_csv('/mnt/data/indian_student_dataset.csv')

df['Percentage'] = df[['Marks in Maths', 'Marks in Science', 'Marks in English']].mean(axis=1)

def assign_grade(percentage):
    if percentage >= 90:
        return 'A+'
    elif percentage >= 80:
        return 'A'
    elif percentage >= 70:
        return 'B'
    elif percentage >= 50:
        return 'C'
    else:
        return 'D'

def assign_status(percentage):
    return 'Pass' if percentage >= 50 else 'Fail'

df['Grade'] = df['Percentage'].apply(assign_grade)
df['Status'] = df['Percentage'].apply(assign_status)

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Student Dashboard", style={'textAlign': 'center', 'color':'green'}),

    # Filters
    html.Div([
        dcc.Dropdown(
            id='class-filter',
            options=[{'label': cls, 'value': cls} for cls in sorted(df['Class'].unique())],
            placeholder="Select Class"
        ),
        dcc.Dropdown(
            id='grade-filter',
            options=[{'label': grade, 'value': grade} for grade in sorted(df['Grade'].unique())],
            placeholder="Select Grade"
        ),
        dcc.Dropdown(
            id='status-filter',
            options=[{'label': status, 'value': status} for status in sorted(df['Status'].unique())],
            placeholder="Select Status"
        )
    ], style={'display': 'flex', 'gap': '10px', 'margin-bottom': '20px'}),

    # Summary Cards
    html.Div([
        html.Div(id='total-students', style={'padding': '10px', 'backgroundColor': '#e0f7fa', 'borderRadius': '10px'}),
        html.Div(id='pass-rate', style={'padding': '10px', 'backgroundColor': '#e8f5e9', 'borderRadius': '10px'}),
        html.Div(id='average-percentage', style={'padding': '10px', 'backgroundColor': '#fff3e0', 'borderRadius': '10px'})
    ], style={'display': 'flex', 'gap': '20px'}),

    # Leaderboard Table
    dash_table.DataTable(
        id='leaderboard-table',
        columns=[{'name': col, 'id': col} for col in ['Student Name', 'Roll Number', 'Class', 'Percentage', 'Grade', 'Status']],
        page_size=10,
        style_data_conditional=[
            {'if': {'filter_query': '{Percentage} >= 90'}, 'backgroundColor': '#d0f0c0'},
            {'if': {'filter_query': '{Percentage} < 50'}, 'backgroundColor': '#ffcdd2'}
        ],
        style_table={'overflowX': 'auto'}
    ),

    # Charts Section
    dcc.Graph(id='grade-distribution'),
    dcc.Graph(id='class-performance'),
    dcc.Graph(id='attendance-vs-percentage'),
    dcc.Graph(id='subject-distribution'),
    dcc.Graph(id='failures-by-class'),
    dcc.Graph(id='status-distribution'),
    dcc.Graph(id='percentage-trend'),
    dcc.Graph(id='subject-performance-box')
])

@app.callback(
    [Output('leaderboard-table', 'data'),
     Output('total-students', 'children'),
     Output('pass-rate', 'children'),
     Output('average-percentage', 'children'),
     Output('grade-distribution', 'figure'),
     Output('class-performance', 'figure'),
     Output('attendance-vs-percentage', 'figure'),
     Output('subject-distribution', 'figure'),
     Output('failures-by-class', 'figure'),
     Output('status-distribution', 'figure'),
     Output('percentage-trend', 'figure'),
     Output('subject-performance-box', 'figure')],
    [Input('class-filter', 'value'),
     Input('grade-filter', 'value'),
     Input('status-filter', 'value')]
)
def update_dashboard(selected_class, selected_grade, selected_status):
    filtered_df = df.copy()

    if selected_class:
        filtered_df = filtered_df[filtered_df['Class'] == selected_class]
    if selected_grade:
        filtered_df = filtered_df[filtered_df['Grade'] == selected_grade]
    if selected_status:
        filtered_df = filtered_df[filtered_df['Status'] == selected_status]

    total_students = f"Total Students: {len(filtered_df)}"
    pass_rate = f"Pass Rate: {filtered_df[filtered_df['Status'] == 'Pass'].shape[0] / len(filtered_df) * 100:.2f}%" if len(filtered_df) > 0 else "Pass Rate: 0.00%"
    average_percentage = f"Average Percentage: {filtered_df['Percentage'].mean():.2f}%" if len(filtered_df) > 0 else "Average Percentage: 0.00%"

    grade_fig = px.pie(filtered_df, names='Grade', title='Grade Distribution')
    class_fig = px.bar(filtered_df, x='Class', y='Percentage', color='Grade', title='Class-wise Performance')
    attendance_fig = px.scatter(filtered_df, x='Attendance', y='Percentage', color='Grade', title='Attendance vs Percentage')
    subject_fig = px.histogram(filtered_df, x=['Marks in Maths', 'Marks in Science', 'Marks in English'], barmode='overlay', title='Subject Marks Distribution')
    failures_fig = px.bar(filtered_df[filtered_df['Status'] == 'Fail'], x='Class', title='Failures by Class')
    status_fig = px.pie(filtered_df, names='Status', title='Status Distribution')
    trend_fig = px.line(filtered_df, x='Roll Number', y='Percentage', title='Percentage Trend over Roll Numbers')
    box_fig = px.box(filtered_df.melt(id_vars=['Student Name'], value_vars=['Marks in Maths', 'Marks in Science', 'Marks in English']), x='variable', y='value', title='Subject Performance Spread')

    return filtered_df.to_dict('records'), total_students, pass_rate, average_percentage, grade_fig, class_fig, attendance_fig, subject_fig, failures_fig, status_fig, trend_fig, box_fig

if __name__ == '__main__':
    app.run_server(debug=True)
